    000_16_230114_141_subway_transer_24
    
    data :024_경강선_주소데이터_20221122.csv
     save : subway_info_24.csv
     display : subway_info_24.html

In [31]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [32]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [33]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [34]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/024_경강선_주소데이터_20221122.csv',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소
0,코레일,경강,경기광주,경기도 광주시 역동 176-1,경기도 광주시 광주역로 10(역동)
1,코레일,경강,곤지암,경기도 광주시 곤지암읍 곤지암리 353-1,경기도 광주시 곤지암천로 333
2,코레일,경강,부발,경기도 이천시 부발읍 아리미 505-7,경기도 이천시 부발읍 신아로 87
3,코레일,경강,삼동,경기도 광주시 삼동 225번지,경기도 광주시 순암로 451
4,코레일,경강,세종대왕릉,경기도 여주시 능서면 신지리 276-2,경기도 여주시 능서면 양화로 877


In [35]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  11 non-null     object
 1   선명       11 non-null     object
 2   역명       11 non-null     object
 3   지번주소     11 non-null     object
 4   도로명주소    11 non-null     object
dtypes: object(5)
memory usage: 568.0+ bytes


In [36]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [37]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [38]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["도로명주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["도로명주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


경기도 여주시 능서면 양화로 877은(는) 검색에 실패했습니다


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y
0,코레일,경강,경기광주,경기도 광주시 역동 176-1,경기도 광주시 광주역로 10(역동),978155.083451217,1933341.48778213
1,코레일,경강,곤지암,경기도 광주시 곤지암읍 곤지암리 353-1,경기도 광주시 곤지암천로 333,986376.158640125,1927957.55516738
2,코레일,경강,부발,경기도 이천시 부발읍 아리미 505-7,경기도 이천시 부발읍 신아로 87,999169.204644001,1917893.91839631
3,코레일,경강,삼동,경기도 광주시 삼동 225번지,경기도 광주시 순암로 451,973745.446218106,1934431.22337524
4,코레일,경강,세종대왕릉,경기도 여주시 능서면 신지리 276-2,경기도 여주시 능서면 양화로 877,NaN,NaN


In [39]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [40]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,경강,경기광주,경기도 광주시 역동 176-1,경기도 광주시 광주역로 10(역동),978155.083451217,1933341.48778213,127.253190,37.398925
1,코레일,경강,곤지암,경기도 광주시 곤지암읍 곤지암리 353-1,경기도 광주시 곤지암천로 333,986376.158640125,1927957.55516738,127.346173,37.350553
2,코레일,경강,부발,경기도 이천시 부발읍 아리미 505-7,경기도 이천시 부발읍 신아로 87,999169.204644001,1917893.91839631,127.490631,37.259939
3,코레일,경강,삼동,경기도 광주시 삼동 225번지,경기도 광주시 순암로 451,973745.446218106,1934431.22337524,127.203331,37.408633
4,코레일,경강,세종대왕릉,경기도 여주시 능서면 신지리 276-2,경기도 여주시 능서면 양화로 877,NaN,NaN,NaN,NaN


In [41]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [42]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [43]:
#5 seoul university 
map

In [44]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_24.html")


In [45]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_24.csv", index = False)
